# Target Accrual Redemption Forward (TARF) Fundamentals

This notebook demonstrates how to access and use the functionalities of **Target Accrual Redemption Forward (TARF)** which are part of our **QPS** module within LSEG Financial Analytics SDK.

**You will be able to:**
* Define TARF instrument with basic parameters
* Configure pricing parameters
* Calculate TARF pricing

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp

import json
import datetime as dt

## Data Preparation

To define a Structured Product instrument, in this example barrier TARF, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, underlying)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

tarf_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "TARF_Target_KO",
    inputs = [
        sp.NameTypeValue(name="Underlying", type = "string", value="USDEUR"),
        sp.NameTypeValue(name="StartDate", type = "date", value= dt.date(2022, 1, 25)),
        sp.NameTypeValue(name="EndDate", type = "string", value="StartDate + 1Y"),
        sp.NameTypeValue(name="Frequency", type = "string", value="3M"),
        sp.NameTypeValue(name="Notional", type = "string", value="100000"),
        sp.NameTypeValue(name="Strike", type = "string", value="0.8"),
        sp.NameTypeValue(name="KnockOutRate", type = "string", value="1.02"),
        sp.NameTypeValue(name="IsKnockOutEvent", type = "string", value="FX[t] >= KnockOutRate"),
        sp.NameTypeValue(name="IsRedemptionEvent", type = "string", value="Sum[t] >= ProfitTarget"),
        sp.NameTypeValue(name="ProfitTarget", type = "string", value="4%"),
        sp.NameTypeValue(name="ResidualProfitTarget", type = "string", value="Max(ProfitTarget - Sum[t-1], 0)"),
        sp.NameTypeValue(name="KO_Payment", type = "string", value="Settlement[t]"),
    ],
    payoff_description = [
					[
                        "Schedule Type",
                        "Schedule description",
                        "FX",
                        "Coupon",
                        "Settlement",
                        "Sum",
                        "Alive",
                        "KO_Amount",
                        "Price"
                    ],
                    [
                        "AtDate",
                        "StartDate",
                        "",
                        "",
                        "0",
                        "",
                        "IF(ProfitTarget >= 0, 1, 0)",
                        "",
                        ""
                    ],
                    [
                        "OnSchedule",
                        "DateTable(StartDate + Frequency, EndDate, Frequency, ResetGap := 0b)",
                        "FxSpot(Underlying)",
                        "IF(IsKnockOutEvent, ResidualProfitTarget, FX[t] - Strike)",
                        "Coupon[t] * Notional",
                        "Sum[LastDate] + max(Coupon[t],0)",
                        "If(IsRedemptionEvent or IsKnockOutEvent, 0, Alive[LastDate-1])",
                        "Alive[LastDate-1] * (1-Alive[LastDate]) * KO_Payment",
                        "Receive (Alive[t] * Settlement[t] + KO_Amount[t])"
                    ],
                    [
                        "AtDate",
                        "EndDate",
                        "FxSpot(Underlying)",
                        "IF(IsKnockOutEvent, ResidualProfitTarget, FX[t] - Strike)",
                        "Coupon[LastDate] * Notional",
                        "Sum[LastDate] + max(Coupon[t],0)",
                        "If(IsRedemptionEvent or IsKnockOutEvent, 0, Alive[LastDate-1])",
                        "Alive[LastDate-1] * (1-Alive[LastDate]) * KO_Payment",
                        "Receive (Alive[LastDate] * Settlement[t] + KO_Amount[t])"
                    ]
				]
)


# 2. Create SP instrument definition object

tarf_target_ko = sp.StructuredProductsDefinitionInstrument(definition = tarf_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

tarf_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2022, 3, 16),  # Set your desired valuation date
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "USDEUR",
            underlying_tag = "USDEUR",
            underlying_currency = "EUR",
            asset_class = "ForeignExchange",
            model_name= "Heston")]
)
print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

response = sp.price(
    definitions=[tarf_target_ko],
    pricing_preferences=tarf_pricing_params,
    market_data=None,
    return_market_data=True,  # or False
    fields=None  # or specify fields as a string
)

print("Pricing execution completed")

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "TARF_Target_KO",
    "dealCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 0.040873302,
        "Coupon": 0.361995511,
        "FX": 3.603602536,
        "KO_Amount": 10459.966529269,
        "Price": 10411.850025818,
        "Settlement": 36199.551141786,
        "Sum": 0.961770256
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}
